# output test result

### 训练词向量 参数 (hs=1,window=7,size=300,min_count=1,workers=4)

In [1]:
from gensim.models import word2vec
from gensim.models import KeyedVectors

# 引入日志配置
#import logging

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 引入数据集
raw_sentences = open('./train-sentences.txt').read()



raw_sentences = raw_sentences.split('\n')

# 切分词汇
sentences = [s.split(' ') for s in raw_sentences]

# 构建模型
model = word2vec.Word2Vec(sentences,hs=1,window=10,size=300,min_count=5,workers =7)

### 准备test上下文

In [9]:
import re
dev = open('./test_set.txt').read()

dev = re.sub(r"\n\n\n",'#', dev)

dev_questions = dev.split('#')
print(len(dev_questions))
dev_questions = dev_questions[0:-1]
print(len(dev_questions))


answer_abcde_dic = {}
question_dic = {}

for dev_question in dev_questions:
    
    question_num = dev_question[:dev_question.find(')')]
    question = dev_question[dev_question.find(' ')+1:dev_question.find('\n')]
    
    
    
    a_answer = dev_question[dev_question.find('a)')+3:dev_question.find('\n',dev_question.find('a)')+3)]
    b_answer = dev_question[dev_question.find('b)')+3:dev_question.find('\n',dev_question.find('b)')+3)]
    c_answer = dev_question[dev_question.find('c)')+3:dev_question.find('\n',dev_question.find('c)')+3)]
    d_answer = dev_question[dev_question.find('d)')+3:dev_question.find('\n',dev_question.find('d)')+3)]
    e_answer = dev_question[dev_question.find('e)')+3:]
    
    question_dic[question_num] = question.replace('_____','')
    
    answer_abcde_dic[question_num] = []
    answer_abcde_dic[question_num].append(a_answer)
    answer_abcde_dic[question_num].append(b_answer)
    answer_abcde_dic[question_num].append(c_answer)
    answer_abcde_dic[question_num].append(d_answer)
    answer_abcde_dic[question_num].append(e_answer)
    
import string
for key in question_dic.keys():
    for letter in string.punctuation:
        question_dic[key] = question_dic[key].replace(letter,' ')
        
# 多个' '变一个' '，去掉前后的' '
for key in question_dic.keys():
    question_dic[key] = re.sub(' +',' ',question_dic[key])
    question_dic[key] = question_dic[key].strip() 
# 变成小写
for key in question_dic.keys():
    question_dic[key] = question_dic[key].lower()

801
800


### 获得预测的概率

In [24]:
def get_pre_wv_pro(question_dic,answer_abcde_dic,model):
    answer_dic = {}

    for key in question_dic.keys():
        
        answer_dic[key] = []
        
        tupeList = model.predict_output_word(question_dic[key].split(' '),topn=40000)
        dicList = {}
        for tup in tupeList:
            dicList[tup[0]] = tup[1]
        
        
        for answer in answer_abcde_dic[key]:
            pro = 0
            if answer in dicList:
                pro = dicList[answer]
                
            answer_dic[key].append(pro)
            
        
            
    answer_list = []
    compare = ['a','b','c','d','e']

    for k,v in answer_dic.items():
        maxIndex = v.index(max(v))
       
        answer_list.append(str(k) + ' ' + str(compare[maxIndex]) + ' ' + answer_abcde_dic[k][maxIndex])
            
    return answer_list


answer_result = get_pre_wv_pro(question_dic,answer_abcde_dic,model)



In [ ]:
f=open('result.txt','w')
for sentence in write_out_list:
    f.write(sentence+"\n")
f.close()